In [13]:
from langchain.document_loaders import DirectoryLoader

doc_loader = DirectoryLoader('./my-app/', glob="**/*.md")

In [14]:
docs = doc_loader.load()

In [15]:
docs = list(filter(lambda x: 'node_modules' not in x.metadata['source'], docs))

the predictions currently are referencing files that they aren't creating. This is a cyclic dependency since
the posts.ts api file might reference a .json or something that wasn't generated by the first prompt

Going to try to update the second prompt to make sure that it's not referencing files that don't exist by 
giving it a list of all the files in the directory and making sure that it will also include other ones.

In [16]:
import os

def list_files(startpath, exclude=[""]):
    outstr = ""
    for root, dirs, files in os.walk(startpath):
        dirs[:] = [d for d in dirs if d not in exclude]
        # print(dirs)
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        outstr += '{}{}/'.format(indent, os.path.basename(root))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            outstr += '{}{}'.format(subindent, f)
    return outstr

In [17]:
dirstring = list_files('./my-app/', set(["node_modules", ".next"]))

In [24]:
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.25, max_tokens=6000, model_name="gpt-4")

template = """
Given the following descriptions of what directories are for and their file conventions,
output all the files needed to add {feature_description} to the app. For each file, include
the description of what the file is for.

Make sure that .ts files will compile.

{markdown_docs}

The reponse format should be a list of file descriptions, like this:
1. path/to/file1.js: in-depth description of file1
...
"""
which_files_prompt = PromptTemplate(
    input_variables=["markdown_docs", "feature_description"],
    template=template,
)

description = """
Create a new view for a user's posts and an API route to return the posts. 
The view should fetch the posts from the API route and display them.
Can just use some dummy data for now from a JSON about cheese. The view should
display the post title and body. The route should return the posts in JSON format.
"""

file_description_chain = LLMChain(prompt=which_files_prompt, llm=llm, verbose=True, output_key="file_descriptions")

# predicted_files = file_description_chain.predict(feature_description=description, directory_description=docs)

In [30]:

fctemplate = """
Given the following files and their descrpitions of what the file is, write the code for the file.
{file_descriptions}

Make sure that any files imported in the code are in the directory structure. If not, add another
file to the output list.

For example, if the code generated includes 
`import {{ posts }} from '../../public/posts'`, then also generate a file called `posts.ts` in the
public directory, because that file is not in the directory structure.

The directory structure: 
{dirstring}

Output: 
File name plus any other files needed to compile the code:
The code for the file
"""

file_content_prompt = PromptTemplate(
    input_variables=["file_descriptions", "dirstring"],
    template=fctemplate
)

file_content_chain = LLMChain(prompt=file_content_prompt, llm=llm, verbose=True, output_key="predictions")
# now we're passing in all the files and their descriptions
# file_content = file_content_chain.predict(file_descriptions=file_descriptions, file_name=file_name, dirstring=dirstring)



In [31]:
from langchain.chains import SequentialChain
# from langchain.memory import SimpleMemory

description = """
Create a new view for a user's posts and an API route to return the posts.
Can just use some dummy data for now from a JSON about cheese. The view should
display the post title and body. The route should return the posts in JSON format.
"""


overall_chain = SequentialChain(
    chains=[file_description_chain, file_content_chain],
    input_variables=["feature_description", "markdown_docs", "dirstring"],
    # Here we return multiple variables
    output_variables=["file_descriptions", "predictions"],
    verbose=True)

In [32]:
predicted_files = overall_chain({"feature_description": description, "markdown_docs": docs, "dirstring": dirstring})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following descriptions of what directories are for and their file conventions,
output all the files needed to add 
Create a new view for a user's posts and an API route to return the posts.
Can just use some dummy data for now from a JSON about cheese. The view should
display the post title and body. The route should return the posts in JSON format.
 to the app. For each file, include
the description of what the file is for.

Make sure that .ts files will compile.

[Document(page_content='This is a Next.js project bootstrapped with create-next-app.\n\nGetting Started\n\nFirst, run the development server:\n\n```bash\nnpm run dev\n\nor\n\nyarn dev\n\nor\n\npnpm dev\n```\n\nOpen http://localhost:3000 with your browser to see the result.\n\nYou can start editing the page by modifying pages/index.tsx. The page auto-updates as you edit the file.\n\nAPI routes can be accessed on ht

In [33]:
# files = predicted_files.strip().split('\n')
# predictions = []
with open('predictions.txt', 'w+') as f:
    f.write(predicted_files["predictions"])
        # file_name, file_description = file.split(':')
        # # pred = predict_file_content(file_description, file_name, dirstring)
        # # make a file with the predicted content
        # # with open(file_name, 'w+') as f:
        # f.write(file_name)
        # f.write(pred)